In [7]:
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from operator import add
from pyspark.ml.linalg import Vectors
from pandas import Series,DataFrame
import pandas as pd
import time
import psutil

In [8]:
def buildArray(listOfIndices):
    
    returnVal = np.zeros(20000)
    
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    
    mysum = np.sum(returnVal)
    
    returnVal = np.divide(returnVal, mysum)
    
    return returnVal

In [9]:
wikiPagesFile="./data/SmallTrainingData.txt"
d_corpus = sc.textFile(wikiPagesFile, 1)
d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')

d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# Next, we get a RDD that has, for each (docID, ["word1", "word2", "word3", ...]),
# ("word1", docID), ("word2", docId), ...
allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

# Now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = d_keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))

# Now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey(lambda x,y: x + y)

# Get the top 20,000 words in a local array in a sorted format based on frequency
# If you want to run it on your laptio, it may a longer time for top 20k words. 
topWords = allCounts.top(20000, lambda x: x[1])
topWords

# We'll create a RDD that has a set of (word, dictNum) pairs
# start by creating an RDD that has the number 0 through 20000
# 20000 is the number of words that will be in our dictionary
topWordsK = sc.parallelize(range(20000))

# Now, we transform (0), (1), (2), ... to ("MostCommonWord", 1)
# ("NextMostCommon", 2), ...
# the number will be the spot in the dictionary used to tell us
# where the word is located
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

# Now join and link them, to get a set of ("word1", (dictionaryPos, docID)) pairs
allDictionaryWords = dictionary.join(allWordsWithDocID)

# allDictionaryWords.take(1)

# Now, we drop the actual word itself to get a set of (docID, dictionaryPos) pairs
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

# Now get a set of (docID, [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

# The following line this gets us a set of
# (docID,  [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
# and converts the dictionary positions to a bag-of-words numpy array...
allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))
# TF

In [4]:
# task 1 -- frequency position

In [5]:
# Frequency position:
print("The frequency position of the words 'applicant' is: ", dictionary.filter(lambda x: x[0] == 'applicant').map(lambda x: x[1]).first())
print("The frequency position of the words 'and' is: ", dictionary.filter(lambda x: x[0] == 'and').map(lambda x: x[1]).first())
print("The frequency position of the words 'attack' is: ", dictionary.filter(lambda x: x[0] == 'attack').map(lambda x: x[1]).first())
print("The frequency position of the words 'protein' is: ", dictionary.filter(lambda x: x[0] == 'protein').map(lambda x: x[1]).first())
print("The frequency position of the words 'car' is: ", dictionary.filter(lambda x: x[0] == 'car').map(lambda x: x[1]).first())

The frequency position of the words 'applicant' is:  346
The frequency position of the words 'and' is:  2
The frequency position of the words 'attack' is:  502
The frequency position of the words 'protein' is:  3014
The frequency position of the words 'car' is:  608


In [7]:
# task 2 learning the model

In [4]:
# rdd_X = allDocsAsNumpyArrays.map(lambda x: np.array(x[1]))
# # rdd_X.take(2)                                 
# rdd_y = allDocsAsNumpyArrays.map(lambda x: np.where(x[0][:2] == 'AU', 1, 0).tolist())   
# # rdd_y.take(2)
# traindata = rdd_y.zip(rdd_X)
# testdata = rdd_y.zip(rdd_X)

In [5]:
# traindata.cache()
# train_size = traindata.count()
# # train_size # 3442

In [6]:
# parameter_size = len(traindata.take(1)[0][1]) + 1
# # parameter_size # 20001

In [7]:
# def LogisticRegression(traindata=traindata,
#                        max_iteration = 100,
#                        learningRate = 0.01,
#                        regularization = 0.01,
#                        precision = 0.01,
#                        optimizer = 'SGD' 
#                       ):

#     # initialization
#     prev_cost = 0
#     L_cost = []
#     prev_validation = 0
#     train_size = traindata.count()

#     parameter_size = len(traindata.take(1)[0][1]) + 1
#     parameter_vector = np.zeros(parameter_size)
# #     parameter_vector = np.zeros(parameter_size) # initialize with zeros
#     momentum = np.zeros(parameter_size)
#     prev_mom = np.zeros(parameter_size)
#     second_mom = np.array(parameter_size)
#     gti = np.zeros(parameter_size)
#     epsilon = 10e-8
    
#     for i in range(max_iteration):

#         bc_weights = parameter_vector[:-1]
#         bc1_weights = parameter_vector[-1]


#         res = traindata.treeAggregate((np.zeros(parameter_size), 0, 0),\
#               lambda x, y:(x[0]\
#                           + (np.append(y[1], 1)) * (-y[0] + (np.exp(np.dot(y[1], bc_weights) + bc1_weights)\
#                           /(1 + np.exp(np.dot(y[1], bc_weights) + bc1_weights)))),\
#                           x[1] \
#                           + y[0] * (-(np.dot(y[1], bc_weights) + bc1_weights)) \
#                           + np.log(1 + np.exp(np.dot(y[1],bc_weights)+ bc1_weights)),\
#                           x[2] + 1),
#               lambda x, y:(x[0] + y[0], x[1] + y[1], x[2] + y[2]))

#         cost =  res[1]

#         # calculate gradients
#         gradient_derivative = (1.0 / res[2]) * res[0]
        
#         if optimizer == 'SGD':
#             parameter_vector = parameter_vector - learningRate * gradient_derivative


#         print("Iteration No.", i, " Cost=", cost)
        
#         # Stop if the cost is not descreasing
#         if abs(cost - prev_cost) < precision:
#             print("cost - prev_cost: " + str(cost - prev_cost))
#             break
#         prev_cost = cost
#         L_cost.append(cost)
        
#     return parameter_vector, L_cost

In [ ]:
# parameter_vector_sgd, L_cost_sgd = LogisticRegression(traindata=traindata,
#                        max_iteration = 100,
#                        learningRate = 0.8,
#                        precision = 0.01,
#                        optimizer = 'SGD' 
#                       )

In [ ]:
# task 2

In [10]:
myRDD = allDocsAsNumpyArrays.map(lambda x: (np.array(x[1]), np.where(x[0][:2] == 'AU', 1, 0)))
# myRDD.take(2)

In [30]:
# Without Regularization
learningRate = 0.001
num_iteration = 100 

myRDD.cache()

allDocsAsNumpyArrays.cache()

precision = 0.01

oldCost = 0

size = allDocsAsNumpyArrays.count()

beta = np.zeros(20000)

cost_list = []

k = 0

for i in range(num_iteration):
    # first map : theta, x, y, e^theta
    gradientCost = myRDD.map(lambda x : ((np.dot(x[0], beta)), x[0], x[1], np.exp(np.dot(x[0], beta))))\
                    .map(lambda x : ((-x[1]) * (x[2] - (x[3]/(1 + x[3]))), (- x[2] * x[0] + np.log(1 + x[3]))))\
                    .reduce(lambda x, y : (x[0] + y[0], x[1] + y[1]))
    
    cost = gradientCost[1]
    
    gradient = gradientCost[0]
    
    cost_list.append(cost)
        
    print(i, " Cost", cost)
    beta = beta - learningRate * gradient
    
    # Stop if the cost is not descreasing 
    if(abs(cost - oldCost) <= precision):
        print("Stoped at iteration", i)
        break
        
    oldCost = cost

0  Cost 2385.812595487336
1  Cost 636.7181733349906
2  Cost 524.7561613504523
3  Cost 477.25189947925145
4  Cost 451.6325149380995
5  Cost 435.9799565004001
6  Cost 425.6190121431654
7  Cost 418.35135615101484
8  Cost 413.01610374106383
9  Cost 408.94823062661834
10  Cost 405.7430454590628
11  Cost 403.14252414854786
12  Cost 400.9759489530843
13  Cost 399.12692041996723
14  Cost 397.5140801670866
15  Cost 396.0793681051971
16  Cost 394.7806152453219
17  Cost 393.58673038569407
18  Cost 392.4744912586729
19  Cost 391.4263571933464
20  Cost 390.42894877041476
21  Cost 389.4719727781179
22  Cost 388.5474503854033
23  Cost 387.6491554501056
24  Cost 386.7722007711947
25  Cost 385.9127299908655
26  Cost 385.0676859184938
27  Cost 384.2346347815908
28  Cost 383.4116318403595
29  Cost 382.5971178907503
30  Cost 381.7898390370957
31  Cost 380.98878413577387
32  Cost 380.19313575703353
33  Cost 379.4022315575973
34  Cost 378.61553372026447
35  Cost 377.8326046795546
36  Cost 377.05308777083695

SyntaxError: 'return' outside function (<ipython-input-30-8664cec7fdbc>, line 43)

In [18]:
# Regularization test
learningRate = 0.001
num_iteration = 400 

myRDD.cache()

allDocsAsNumpyArrays.cache()

precision = 0.05

oldCost = 0

size = allDocsAsNumpyArrays.count()

beta = np.zeros(20000)

cost_list_r = []

k = 0

for i in range(num_iteration):
    # first map : theta, x, y, e^theta
    gradientCost = myRDD.map(lambda x : ((np.dot(x[0], beta)), x[0], x[1], np.exp(np.dot(x[0], beta))))\
                    .map(lambda x : ((-x[1]) * (x[2] - (x[3]/(1 + x[3]))), (- x[2] * x[0] + np.log(1 + x[3]))))\
                    .reduce(lambda x, y : (x[0] + y[0], x[1] + y[1]))
    
    cost = gradientCost[1]
    
    # 0.001 as regulazition penalty
    gradient = gradientCost[0] + 0.001 * np.linalg.norm(beta)
    
    cost_list_r.append(cost)
        
    if i%10 == 0 :
            print("Iteration No.", i, " Cost=", cost)
            
    beta = beta - learningRate * gradient
    
    # Stop if the cost is not descreasing       
    if(abs(cost - oldCost) <= precision):
        print("Stoped at iteration", i)
        break
        
    oldCost = cost

Iteration No. 0  Cost= 2385.812595487336
Iteration No. 10  Cost= 2096.2079361716783
Iteration No. 20  Cost= 1861.1496601474453
Iteration No. 30  Cost= 1669.402351331815
Iteration No. 40  Cost= 1511.8788883596835
Iteration No. 50  Cost= 1381.4133840652987
Iteration No. 60  Cost= 1272.4278668836914
Iteration No. 70  Cost= 1180.601398037059
Iteration No. 80  Cost= 1102.585362867379
Iteration No. 90  Cost= 1035.7748877365136
Iteration No. 100  Cost= 978.1318587268526
Iteration No. 110  Cost= 928.0504366757368
Iteration No. 120  Cost= 884.2557334103205
Iteration No. 130  Cost= 845.7276504882192
Iteration No. 140  Cost= 811.6435443992552
Iteration No. 150  Cost= 781.3348942344497
Iteration No. 160  Cost= 754.2543738455296
Iteration No. 170  Cost= 729.9506716678634
Iteration No. 180  Cost= 708.0491036111616
Iteration No. 190  Cost= 688.2365807535407
Iteration No. 200  Cost= 670.2498706611949
Iteration No. 210  Cost= 653.8663660213707
Iteration No. 220  Cost= 638.8967748602693
Iteration No. 23

In [16]:
# Print out the five words with the largest regression coefficients
d = beta.argsort()[-5:][::-1]
for i in range(0, 5):
    print(dictionary.filter(lambda x : (x[1] == d[i])).collect())

[('applicant', 346)]
[('tribunal', 485)]
[('respondent', 709)]
[('mr', 206)]
[('appellant', 1372)]


In [ ]:
d = beta.argsort()[-5:][::-1]
for i in range(0, 5):
    print(dictionary.filter(lambda x : (x[1] == d[i])).collect())

In [36]:
# Task 3 evaluation of learned model
test = "./data/TestingData.txt"
d_corpus_t = sc.textFile(test, 1)
d_keyAndText_t = d_corpus_t.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')

d_keyAndListOfWords_t = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# Next, we get a RDD that has, for each (docID, ["word1", "word2", "word3", ...]),
# ("word1", docID), ("word2", docId), ...
allWordsWithDocID_t = d_keyAndListOfWords_t.flatMap(lambda x: ((j, x[0]) for j in x[1]))

# Now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords_t = d_keyAndListOfWords_t.flatMap(lambda x: x[1]).map(lambda x: (x, 1))

# Now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts_t = allWords_t.reduceByKey(lambda x,y: x + y)

# Get the top 20,000 words in a local array in a sorted format based on frequency
# If you want to run it on your laptio, it may a longer time for top 20k words. 
topWords_t = allCounts_t.top(20000, lambda x: x[1])

# We'll create a RDD that has a set of (word, dictNum) pairs
# start by creating an RDD that has the number 0 through 20000
# 20000 is the number of words that will be in our dictionary
topWordsK_t = sc.parallelize(range(20000))

# Now, we transform (0), (1), (2), ... to ("MostCommonWord", 1)
# ("NextMostCommon", 2), ...
# the number will be the spot in the dictionary used to tell us
# where the word is located
dictionary_t = topWordsK_t.map (lambda x : (topWords[x][0], x))

# Now join and link them, to get a set of ("word1", (dictionaryPos, docID)) pairs
allDictionaryWords_t = dictionary_t.join(allWordsWithDocID)

# allDictionaryWords.take(1)

# Now, we drop the actual word itself to get a set of (docID, dictionaryPos) pairs
justDocAndPos_t = allDictionaryWords_t.map(lambda x: (x[1][1], x[1][0]))

# Now get a set of (docID, [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
allDictionaryWordsInEachDoc_t = justDocAndPos_t.groupByKey()

# The following line this gets us a set of
# (docID,  [dictionaryPos1, dictionaryPos2, dictionaryPos3...]) pairs
# and converts the dictionary positions to a bag-of-words numpy array...
allDocsAsNumpyArrays_t = allDictionaryWordsInEachDoc_t.map(lambda x: (x[0], buildArray(x[1])))
# TF